In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [2]:
DTYPE = 'float64'
L63_data_path = '../data/L63-trajectories'
save_folder='../data/adaptive-rate'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(2e5)
save_interval = 100
N = 2000
L0 = 0.4
L1 = 3.5
beta = 4e-5
min_change = (1.-0.1)**(1./log_interval) - 1.
train = np.load(f'{L63_data_path}/train.npy').astype(DTYPE)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(DTYPE)

model = srnn.SurrogateModel_NN(3, 100, name='nn', save_folder=save_folder)
model.learn(train[:, :N], steps, 1e-3, batch_size='GD', log_interval=100, save_interval=1000,\
           max_delta=0.1, min_change=min_change, constant_rate=False, update_frequency=1000, mode='BS')
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
tau_f_1 = tau_f_se.mean()
tau_f_1

step: 0 loss: 1532905.759432 time: 0.0023 lr: 0.001000,  change: nan%
step: 100 loss: 721707.193045 time: 0.1123 lr: 0.001000,  change: -52.92%
step: 200 loss: 348314.172138 time: 0.2181 lr: 0.001000,  change: -51.74%
step: 300 loss: 217595.944136 time: 0.3257 lr: 0.001000,  change: -37.53%
step: 400 loss: 166429.778360 time: 0.4317 lr: 0.001000,  change: -23.51%
step: 500 loss: 131424.075519 time: 0.5380 lr: 0.001000,  change: -21.03%
step: 600 loss: 102391.867849 time: 0.6470 lr: 0.001000,  change: -22.09%
step: 700 loss: 76715.019831 time: 0.7535 lr: 0.001000,  change: -25.08%
step: 800 loss: 54929.317465 time: 0.8599 lr: 0.001000,  change: -28.40%
step: 900 loss: 36955.979627 time: 0.9676 lr: 0.001000,  change: -32.72%
step: 1000 loss: 25606.315954 time: 1.0737 lr: 0.001000,  change: -30.71%
step: 1100 loss: 18271.649909 time: 1.1807 lr: 0.001000,  change: -28.64%
step: 1200 loss: 13609.173795 time: 1.2871 lr: 0.001000,  change: -25.52%
step: 1300 loss: 10577.999438 time: 1.4019 lr

0.9560459999999997